# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [4]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
short_cities = ['paris', 'minneapolis', 'new york']

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

628

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [23]:
    
def convert_k_to_f(ktemp):
    return (9/5.0) * (ktemp - 273.0) + 32.0


for i, city in enumerate(cities):
    print(f'{i}: {city}')
    ucity = city.replace(" ", "%20")
    try: 
        url = f'http://api.openweathermap.org/data/2.5/weather?q={ucity}&appid={weather_api_key}'
        data = requests.get(url).json()
        weather_desc = data["weather"][0]["description"]
        feels_like = convert_k_to_f(float(data["main"]["feels_like"]))
        print(f'Weather description: {weather_desc}')
        print(f'Temp feels like: {feels_like:.2f}')
        print()
        time.sleep(1)
    except:
        print(f'No data found for {city}')
        print('')

        

0: waingapu
Weather description: scattered clouds
Temp feels like: 71.96

1: pundaguitan
Weather description: moderate rain
Temp feels like: 88.59

2: solsvik
No data found for solsvik

3: vaini
Weather description: scattered clouds
Temp feels like: 71.37

4: ushuaia
Weather description: overcast clouds
Temp feels like: 25.97

5: terrace
Weather description: overcast clouds
Temp feels like: 57.72

6: saint-philippe
Weather description: overcast clouds
Temp feels like: 52.20

7: avarua
Weather description: light rain
Temp feels like: 78.76

8: bluff
Weather description: light rain
Temp feels like: 34.36

9: oistins
Weather description: scattered clouds
Temp feels like: 86.43

10: grand river south east
No data found for grand%20river%20south%20east

11: tuktoyaktuk
Weather description: scattered clouds
Temp feels like: 39.07

12: amderma
No data found for amderma

13: babayurt
Weather description: scattered clouds
Temp feels like: 72.36

14: ambodifototra
No data found for ambodifototra

No data found for vaitupu

121: bafq
Weather description: clear sky
Temp feels like: 73.72

122: kruisfontein
Weather description: clear sky
Temp feels like: 50.50

123: treinta y tres
Weather description: overcast clouds
Temp feels like: 68.86

124: sola
Weather description: broken clouds
Temp feels like: 74.30

125: barentsburg
No data found for barentsburg

126: trat
Weather description: few clouds
Temp feels like: 82.33

127: coquimbo
Weather description: scattered clouds
Temp feels like: 49.50

128: kapaa
Weather description: moderate rain
Temp feels like: 83.14

129: prince rupert
Weather description: overcast clouds
Temp feels like: 55.87

130: mount gambier
Weather description: overcast clouds
Temp feels like: 39.29

131: lolua
No data found for lolua

132: sao jose da coroa grande
Weather description: few clouds
Temp feels like: 75.60

133: quang ngai
Weather description: overcast clouds
Temp feels like: 81.99

134: uhlove
Weather description: clear sky
Temp feels like: 70.99


Weather description: broken clouds
Temp feels like: 44.89

240: honiara
Weather description: scattered clouds
Temp feels like: 80.22

241: mount isa
Weather description: clear sky
Temp feels like: 45.86

242: kavaratti
Weather description: light rain
Temp feels like: 85.68

243: byron bay
Weather description: broken clouds
Temp feels like: 47.07

244: haines junction
Weather description: light rain
Temp feels like: 59.56

245: baykit
Weather description: overcast clouds
Temp feels like: 53.35

246: nome
Weather description: overcast clouds
Temp feels like: 50.49

247: pisco
Weather description: clear sky
Temp feels like: 65.75

248: soyo
Weather description: broken clouds
Temp feels like: 73.76

249: alice springs
Weather description: clear sky
Temp feels like: 28.49

250: wahran
No data found for wahran

251: vanavara
Weather description: scattered clouds
Temp feels like: 49.10

252: victoria
Weather description: broken clouds
Temp feels like: 90.86

253: manokwari
Weather description

Weather description: overcast clouds
Temp feels like: 57.38

359: chenzhou
Weather description: clear sky
Temp feels like: 81.66

360: vilya
Weather description: clear sky
Temp feels like: 56.68

361: port arthur
Weather description: scattered clouds
Temp feels like: 75.24

362: taburao
No data found for taburao

363: ostersund
Weather description: broken clouds
Temp feels like: 34.72

364: montepuez
Weather description: broken clouds
Temp feels like: 60.46

365: souillac
Weather description: broken clouds
Temp feels like: 61.47

366: greencastle
Weather description: broken clouds
Temp feels like: 88.48

367: kavieng
Weather description: scattered clouds
Temp feels like: 84.81

368: sur
Weather description: broken clouds
Temp feels like: 82.96

369: tumannyy
No data found for tumannyy

370: durango
Weather description: moderate rain
Temp feels like: 74.57

371: harper
Weather description: overcast clouds
Temp feels like: 77.90

372: faanui
Weather description: scattered clouds
Temp fee

No data found for ust-kamchatsk

477: brawley
Weather description: clear sky
Temp feels like: 100.63

478: komsomolskiy
Weather description: clear sky
Temp feels like: 73.29

479: kargopol
Weather description: overcast clouds
Temp feels like: 54.86

480: frontera
Weather description: broken clouds
Temp feels like: 96.66

481: manerbio
Weather description: heavy intensity rain
Temp feels like: 69.49

482: hofn
Weather description: broken clouds
Temp feels like: 49.08

483: salinas
Weather description: scattered clouds
Temp feels like: 63.68

484: tilichiki
Weather description: overcast clouds
Temp feels like: 49.10

485: nayoro
Weather description: overcast clouds
Temp feels like: 62.53

486: sarh
Weather description: overcast clouds
Temp feels like: 80.51

487: esil
Weather description: overcast clouds
Temp feels like: 60.13

488: bandar-e torkaman
No data found for bandar-e%20torkaman

489: porto empedocle
Weather description: clear sky
Temp feels like: 69.84

490: narsaq
Weather desc

No data found for chengmai

595: yelizovo
Weather description: scattered clouds
Temp feels like: 46.06

596: montrose
Weather description: scattered clouds
Temp feels like: 55.72

597: sibu
Weather description: scattered clouds
Temp feels like: 84.43

598: arua
Weather description: light rain
Temp feels like: 65.30

599: plaster rock
Weather description: clear sky
Temp feels like: 76.37

600: djibo
Weather description: scattered clouds
Temp feels like: 93.83

601: wulanhaote
No data found for wulanhaote

602: mabaruma
Weather description: light rain
Temp feels like: 88.27

603: unai
Weather description: clear sky
Temp feels like: 71.69

604: humaita
Weather description: overcast clouds
Temp feels like: 92.84

605: zhuhai
Weather description: few clouds
Temp feels like: 95.50

606: kegayli
No data found for kegayli

607: umzimvubu
No data found for umzimvubu

608: sitka
Weather description: overcast clouds
Temp feels like: 57.20

609: sovetskiy
Weather description: clear sky
Temp feels 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [31]:
#Temperature (F) vs. Latitude
#Humidity (%) vs. Latitude
#Cloudiness (%) vs. Latitude
#Wind Speed (mph) vs. Latitude

def build_row(city):
    try: 
        ucity = city.replace(" ", "%20")
        url = f'http://api.openweathermap.org/data/2.5/weather?q={ucity}&appid={weather_api_key}'
        print(url)
        data = requests.get(url).json()
        # Get necessary data for df
        lon = data["coord"]["lon"]
        lat = data["coord"]["lat"]
        ftemp = convert_k_to_f(float(data["main"]["temp"]))
        humidity = data["main"]["humidity"]
        cloudiness = data["clouds"]["all"]
        wind_speed = data["wind"]["speed"]
        time.sleep(1)
        return {"City" : city,
               "Longitude": lon,
               "Latitude" : lat,
               "Temperature" : ftemp,
               "Humidity" : humidity,
               "Cloudiness" : cloudiness,
               "Wind Speed" : wind_speed}
    except:
        return 0
    
    
city_df = pd.DataFrame(columns=['City', 'Longitude', 'Latitude', 'Temperature', 'Humidity', 'Cloudiness', 'Wind Speed'])
row = build_row("paris")
if row != 0:
    city_df.append(row, ignore_index=True)
city_df

http://api.openweathermap.org/data/2.5/weather?q=paris&appid=8faf8df6a0906553e13e8a6689203f78


,City,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed


In [27]:
sample_dict = {"City" : ["Minneapolis", "Milwaukee"],
               "Longitude": [1, 2],
               "Latitude" : [1, 2],
               "Temperature" : [75, 80],
               "Humidity" : [10, 52],
               "Cloudiness" : [20, 21],
               "Wind Speed" : [3.6, 4.2]}
sample_df = pd.DataFrame(sample_dict)
sample_df = sample_df.append({"City" : "Seattle",
               "Longitude": 1,
               "Latitude" : 5,
               "Temperature" : 77,
               "Humidity" : 52,
               "Cloudiness" : 20,
               "Wind Speed" : 3.7}, ignore_index=True)
sample_df

,City,Longitude,Latitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Minneapolis,1,1,75,10,20,3.6
1,Milwaukee,2,2,80,52,21,4.2
2,Seattle,1,5,77,52,20,3.7


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression